In [2]:
import pandas as pd

In [3]:
from transformers import pipeline

In [4]:
from transformers import logging
logging.set_verbosity_error()

In [5]:
classifier = pipeline("zero-shot-classification", device=0)

## Get the Data

In [16]:
tweets = pd.read_csv('tweets.csv', encoding='ISO-8859-1')

In [17]:
df_head = tweets.head(30)

In [18]:
samples = tweets[1:50]

In [19]:
strings = samples.tweet_text.values

In [20]:
sequence = list(strings)

# Sentiment analysis

In [21]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
CUDA_LAUNCH_BLOCKING=1
torch.cuda.set_device(0)
torch.cuda.empty_cache()
PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb=2000

In [22]:
sentiment_labels = ['negative','positive']


In [23]:
sent = classifier(sequence, sentiment_labels)

In [24]:
sent

[{'sequence': 'This phase 2 Acalabrutinib-Venetoclax (AV) trial that is still in recruitment phase will study how well venetoclax and acalabrutinib works in MCL patients who either relapsed or non-respondent to the initial therapy.\r\n\r\nhttps://t.co/gg0G9At23N',
  'labels': ['positive', 'negative'],
  'scores': [0.6042463779449463, 0.3957536518573761]},
 {'sequence': '#NICE backs #AstraZenecas #Calquence for #CLL https://t.co/Vb5lPDoGrA',
  'labels': ['positive', 'negative'],
  'scores': [0.9937306046485901, 0.006269391160458326]},
 {'sequence': '#acalabrutinib is a valuable option in pts intolerant to #ibrutinib. Further valuable data to help decision making in #CLL \r\n\r\nEarly View | Haematologica https://t.co/Z2kCLZaX0D',
  'labels': ['positive', 'negative'],
  'scores': [0.9003549814224243, 0.09964502602815628]},
 {'sequence': 'NICE has recommended the use of acalabrutinib for patients with treatment-naÃ¯ve chronic lymphocytic leukemia. Find out more here https://t.co/6OuJptLCI

### Converting the result into DataFrame with the maximum score

In [25]:
output = []

filter_key1 = ['labels']
filter_key2 = ['scores']

for index, row in df_head.iterrows():
  d = {}
  seq = row['tweet_text']
  result = classifier(seq, sentiment_labels)
 # print(result)
  temp_label = list(map(result.get, filter_key1))
  temp_score = list(map(result.get, filter_key2))
 # print(temp)
  d['tweet_text'] = row['tweet_text']
  d['label'] = temp_label[0][0]
  d['score'] = temp_score[0][0]
  output.append(d) 


In [26]:
 new = pd.DataFrame(output)
 new

,tweet_text,label,score
0,âï¸ Scientists conducted a Phase II study o...,positive,0.867832
1,This phase 2 Acalabrutinib-Venetoclax (AV) tri...,positive,0.604246
2,#NICE backs #AstraZenecas #Calquence for #CLL ...,positive,0.993731
3,#acalabrutinib is a valuable option in pts int...,positive,0.900355
4,NICE has recommended the use of acalabrutinib ...,positive,0.902079
5,NICE backs AstraZenecaâs Calquence for CLL h...,positive,0.971618
6,This is England for now - these decisions usua...,negative,0.881994
7,"AstraZenecaâs Calquence (acalabrutinib), a c...",negative,0.775089
8,Superstar @tobyeyre82 responding to the excell...,positive,0.994777
9,CLL patients all know the drug Ibrutinib and y...,negative,0.886406


In [27]:
new.to_csv('objective2.csv')